In [1]:
from pyspark.sql import SparkSession, functions as F

# Cell to create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 ASSIGNMENT 1 DUSTIN")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/13 19:24:21 WARN Utils: Your hostname, DESKTOP-3ADPNV0 resolves to a loopback address: 127.0.1.1; using 192.168.150.145 instead (on interface eth0)
22/08/13 19:24:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/13 19:24:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
yellow = spark.read.parquet('../../mast30034-project-1-dustintano10/data/raw/yellow_taxi/')
green = spark.read.parquet('../../mast30034-project-1-dustintano10/data/raw/green_taxi/')

In [3]:
yellow.show(1, vertical=True, truncate=100)
yellow.printSchema()

-RECORD 0------------------------------------
 VendorID              | 1                   
 tpep_pickup_datetime  | 2018-10-01 00:23:34 
 tpep_dropoff_datetime | 2018-10-01 00:44:50 
 passenger_count       | 1.0                 
 trip_distance         | 6.2                 
 RatecodeID            | 1.0                 
 store_and_fwd_flag    | N                   
 PULocationID          | 68                  
 DOLocationID          | 7                   
 payment_type          | 2                   
 fare_amount           | 20.5                
 extra                 | 0.5                 
 mta_tax               | 0.5                 
 tip_amount            | 0.0                 
 tolls_amount          | 0.0                 
 improvement_surcharge | 0.3                 
 total_amount          | 21.8                
 congestion_surcharge  | null                
 airport_fee           | null                
only showing top 1 row

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pic

In [4]:
# separates the date and time from both the pickup and dropoff date time columns
# creates new columns for pickup/dropoff date and pickup/dropoff time

from pyspark.sql.functions import *
yellow = yellow.withColumn("pickup_date",
                 to_date(col("tpep_pickup_datetime"),"yyyy-MM-dd"))

yellow = yellow.withColumn("dropoff_date",
                 to_date(col("tpep_pickup_datetime"),"yyyy-MM-dd"))

yellow = yellow.withColumn("pickup_time", date_format('tpep_pickup_datetime', 'HH:mm:ss'))

yellow = yellow.withColumn("dropoff_time", date_format('tpep_dropoff_datetime', 'HH:mm:ss'))

In [6]:
# Remove columns I deemed irrelevant
# For congestions_surcharge,airport_fee,VendorID and passenger_count I see them as irrelevant due to the fact 
# they don't really contribute to what I am focusing in which is the taxi profits, tipping and location popularity
# For tpep_pickup_datetime and tpep_dropoff_datetime, I created new columns for them.

yellow = yellow.drop('congestion_surcharge','airport_fee','tpep_pickup_datetime','tpep_dropoff_datetime','VendorID'
                     ,'passenger_count')

yellow.filter(F.col('payment_type') == 1).limit(10)

22/08/13 19:26:06 ERROR Executor: Exception in task 7.0 in stage 3.0 (TID 10)
org.apache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file file:///mnt/c/Users/User/notebook/jupyterenv/mast30034-project-1-dustintano10/data/raw/yellow_taxi/2019-02.parquet. Column: [congestion_surcharge], Expected: int, Found: DOUBLE
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:706)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:278)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:553)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCode

22/08/13 19:26:06 WARN TaskSetManager: Lost task 3.0 in stage 3.0 (TID 6) (192.168.150.145 executor driver): TaskKilled (Stage cancelled)
22/08/13 19:26:06 WARN TaskSetManager: Lost task 2.0 in stage 3.0 (TID 5) (192.168.150.145 executor driver): TaskKilled (Stage cancelled)
22/08/13 19:26:06 WARN TaskSetManager: Lost task 8.0 in stage 3.0 (TID 11) (192.168.150.145 executor driver): TaskKilled (Stage cancelled)
22/08/13 19:26:06 WARN TaskSetManager: Lost task 1.0 in stage 3.0 (TID 4) (192.168.150.145 executor driver): TaskKilled (Stage cancelled)
22/08/13 19:26:07 WARN TaskSetManager: Lost task 0.0 in stage 3.0 (TID 3) (192.168.150.145 executor driver): TaskKilled (Stage cancelled)


Py4JJavaError: An error occurred while calling o66.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 3.0 failed 1 times, most recent failure: Lost task 6.0 in stage 3.0 (TID 9) (192.168.150.145 executor driver): org.apache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file file:///mnt/c/Users/User/notebook/jupyterenv/mast30034-project-1-dustintano10/data/raw/yellow_taxi/2019-04.parquet. Column: [congestion_surcharge], Expected: int, Found: DOUBLE
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:706)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:278)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:553)
	at org.apache.spark.sql.catalyst.expressi

22/08/13 19:26:10 ERROR Executor: Exception in task 5.0 in stage 4.0 (TID 17)
org.apache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file file:///mnt/c/Users/User/notebook/jupyterenv/mast30034-project-1-dustintano10/data/raw/yellow_taxi/2019-01.parquet. Column: [congestion_surcharge], Expected: int, Found: DOUBLE
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:706)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:278)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:553)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCode

Py4JJavaError: An error occurred while calling o66.getRowsToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 4.0 failed 1 times, most recent failure: Lost task 5.0 in stage 4.0 (TID 17) (192.168.150.145 executor driver): org.apache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file file:///mnt/c/Users/User/notebook/jupyterenv/mast30034-project-1-dustintano10/data/raw/yellow_taxi/2019-01.parquet. Column: [congestion_surcharge], Expected: int, Found: DOUBLE
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:706)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:278)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:553)
	at org.apache.spark.sql.catalyst.ex

22/08/13 19:26:11 WARN TaskSetManager: Lost task 3.0 in stage 4.0 (TID 15) (192.168.150.145 executor driver): TaskKilled (Stage cancelled)
22/08/13 19:26:11 WARN TaskSetManager: Lost task 0.0 in stage 4.0 (TID 12) (192.168.150.145 executor driver): TaskKilled (Stage cancelled)
